# Importar librerías

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

# Cargar datos originales

In [3]:
# Cargar bases de datos
po = pd.read_csv("./data/pokemon.csv")
co = pd.read_csv("./data/combats.csv")

print("Forma del DataFrame pokemon:", po.shape)
print("\nPrimeras filas del DataFrame pokemon:")
print(po.head())

print("\nForma del DataFrame combats:", co.shape)
print("\nPrimeras filas del DataFrame combats:")
print(co.head())

Forma del DataFrame pokemon: (800, 12)

Primeras filas del DataFrame pokemon:
   #           Name Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
0  1      Bulbasaur  Grass  Poison  45      49       49       65       65   
1  2        Ivysaur  Grass  Poison  60      62       63       80       80   
2  3       Venusaur  Grass  Poison  80      82       83      100      100   
3  4  Mega Venusaur  Grass  Poison  80     100      123      122      120   
4  5     Charmander   Fire     NaN  39      52       43       60       50   

   Speed  Generation  Legendary  
0     45           1      False  
1     60           1      False  
2     80           1      False  
3     80           1      False  
4     65           1      False  

Forma del DataFrame combats: (50000, 3)

Primeras filas del DataFrame combats:
   First_pokemon  Second_pokemon  Winner
0            266             298     298
1            702             701     701
2            191             668     668
3          

# Creación de dataset unificado (ejecutar solo 1 vez)

In [ ]:
po.rename(columns={'#':'id', 
                   'Type 1':'Type_1',
                    'Type 2':'Type_2',
                    'Sp. Atk':'Sp_Atk',
                    'Sp. Def': 'Sp_Def'}, inplace=True)
po.columns
po.dtypes

merge_df = co.merge(
    po.add_prefix("first_"),
    left_on="First_pokemon",
    right_on="first_id",
    how="left"
)

merge_df = merge_df.merge(
    po.add_prefix("second_"),
    left_on="Second_pokemon",
    right_on="second_id",
    how="left"
)

merge_df.drop(columns=["First_pokemon", "Second_pokemon"], inplace=True)

merge_df.to_csv("pokemon_battles_full.csv", index=False)


# Lectura del nuevo dataset

In [8]:
df = pd.read_csv("./data/pokemon_battles_full.csv")

#Eliminar columnas que no tienen función en este programa (informativas)
df.drop(columns=["first_Generation", "first_Legendary", "second_Generation", "second_Legendary"], inplace=True)

df.head()

,Winner,first_id,first_Name,first_Type_1,first_Type_2,first_HP,first_Attack,first_Defense,first_Sp_Atk,first_Sp_Def,...,second_id,second_Name,second_Type_1,second_Type_2,second_HP,second_Attack,second_Defense,second_Sp_Atk,second_Sp_Def,second_Speed
0,298,266,Larvitar,Rock,Ground,50,64,50,45,50,...,298,Nuzleaf,Grass,Dark,70,70,40,60,40,60
1,701,702,Virizion,Grass,Fighting,91,90,72,90,129,...,701,Terrakion,Rock,Fighting,91,129,90,72,90,108
2,668,191,Togetic,Fairy,Flying,55,40,85,80,105,...,668,Beheeyem,Psychic,NaN,75,75,75,125,95,40
3,683,237,Slugma,Fire,NaN,40,40,40,70,40,...,683,Druddigon,Dragon,NaN,77,120,90,60,90,48
4,151,151,Omastar,Rock,Water,70,60,125,115,70,...,231,Shuckle,Bug,Rock,20,10,230,10,230,5


# Ingeniería de características
Crear nuevas columnas con datos importantes para el modelo

In [9]:
# Diccionario completo de tipos Pokémon y multiplicadores de daño
# Formato: {tipo_atacante: {tipo_defensor: multiplicador}}

tipos_pokemon = {
    "Normal": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 0.5, "Fantasma": 0.0, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 1.0
    },
    "Fuego": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 0.5, "Eléctrico": 1.0, "Planta": 2.0,
        "Hielo": 2.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 2.0, "Roca": 0.5, "Fantasma": 1.0, "Dragón": 0.5,
        "Siniestro": 1.0, "Acero": 2.0, "Hada": 1.0
    },
    "Agua": {
        "Normal": 1.0, "Fuego": 2.0, "Agua": 0.5, "Eléctrico": 1.0, "Planta": 0.5,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 2.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 2.0, "Fantasma": 1.0, "Dragón": 0.5,
        "Siniestro": 1.0, "Acero": 1.0, "Hada": 1.0
    },
    "Eléctrico": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 2.0, "Eléctrico": 0.5, "Planta": 0.5,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 0.0, "Volador": 2.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 0.5,
        "Siniestro": 1.0, "Acero": 1.0, "Hada": 1.0
    },
    "Planta": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 2.0, "Eléctrico": 1.0, "Planta": 0.5,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 0.5, "Tierra": 2.0, "Volador": 0.5,
        "Psíquico": 1.0, "Bicho": 0.5, "Roca": 2.0, "Fantasma": 1.0, "Dragón": 0.5,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 1.0
    },
    "Hielo": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 0.5, "Eléctrico": 1.0, "Planta": 2.0,
        "Hielo": 0.5, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 2.0, "Volador": 2.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 2.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 1.0
    },
    "Lucha": {
        "Normal": 2.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 2.0, "Lucha": 1.0, "Veneno": 0.5, "Tierra": 1.0, "Volador": 0.5,
        "Psíquico": 0.5, "Bicho": 0.5, "Roca": 2.0, "Fantasma": 0.0, "Dragón": 1.0,
        "Siniestro": 2.0, "Acero": 2.0, "Hada": 0.5
    },
    "Veneno": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 2.0,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 0.5, "Tierra": 0.5, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 0.5, "Fantasma": 0.5, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 0.0, "Hada": 2.0
    },
    "Tierra": {
        "Normal": 1.0, "Fuego": 2.0, "Agua": 1.0, "Eléctrico": 2.0, "Planta": 0.5,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 2.0, "Tierra": 1.0, "Volador": 0.0,
        "Psíquico": 1.0, "Bicho": 0.5, "Roca": 2.0, "Fantasma": 1.0, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 2.0, "Hada": 1.0
    },
    "Volador": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 0.5, "Planta": 2.0,
        "Hielo": 1.0, "Lucha": 2.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 2.0, "Roca": 0.5, "Fantasma": 1.0, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 1.0
    },
    "Psíquico": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 2.0, "Veneno": 2.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 0.5, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 1.0,
        "Siniestro": 0.0, "Acero": 0.5, "Hada": 1.0
    },
    "Bicho": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 2.0,
        "Hielo": 1.0, "Lucha": 0.5, "Veneno": 0.5, "Tierra": 1.0, "Volador": 0.5,
        "Psíquico": 2.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 0.5, "Dragón": 1.0,
        "Siniestro": 2.0, "Acero": 0.5, "Hada": 0.5
    },
    "Roca": {
        "Normal": 1.0, "Fuego": 2.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 2.0, "Lucha": 0.5, "Veneno": 1.0, "Tierra": 0.5, "Volador": 2.0,
        "Psíquico": 1.0, "Bicho": 2.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 1.0
    },
    "Fantasma": {
        "Normal": 0.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 2.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 2.0, "Dragón": 1.0,
        "Siniestro": 0.5, "Acero": 1.0, "Hada": 1.0
    },
    "Dragón": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 2.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 0.0
    },
    "Siniestro": {
        "Normal": 1.0, "Fuego": 1.0, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 0.5, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 2.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 2.0, "Dragón": 1.0,
        "Siniestro": 0.5, "Acero": 1.0, "Hada": 0.5
    },
    "Acero": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 0.5, "Eléctrico": 0.5, "Planta": 1.0,
        "Hielo": 2.0, "Lucha": 1.0, "Veneno": 1.0, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 2.0, "Fantasma": 1.0, "Dragón": 1.0,
        "Siniestro": 1.0, "Acero": 0.5, "Hada": 2.0
    },
    "Hada": {
        "Normal": 1.0, "Fuego": 0.5, "Agua": 1.0, "Eléctrico": 1.0, "Planta": 1.0,
        "Hielo": 1.0, "Lucha": 2.0, "Veneno": 0.5, "Tierra": 1.0, "Volador": 1.0,
        "Psíquico": 1.0, "Bicho": 1.0, "Roca": 1.0, "Fantasma": 1.0, "Dragón": 2.0,
        "Siniestro": 2.0, "Acero": 0.5, "Hada": 1.0
    }
}

# Correr una sola vez

In [11]:
def calcular_ventaja(tipos_atacante, tipos_defensor):
    """
    tipos_atacante: lista de 1 o 2 tipos (ej: ['Agua', 'Tierra'])
    tipos_defensor: lista de 1 o 2 tipos (ej: ['Planta'])
    """
    ventajas = []

    for tipo_atacante in tipos_atacante:
        if pd.isna(tipo_atacante):
            continue
        total_ventaja = 1.0
        for tipo_defensor in tipos_defensor:
            if pd.isna(tipo_defensor):
                continue
            efectividad = tipos_pokemon.get(tipo_atacante, {}).get(tipo_defensor, 1.0)
            total_ventaja *= efectividad
        ventajas.append(total_ventaja)

    # Si alguno da 0 pero hay otro >0, ignoramos el 0
    ventajas_filtradas = [v for v in ventajas if v > 0]
    if ventajas_filtradas:
        return round(max(ventajas_filtradas), 2)
    else:
        return 0.0

# --- Aplicar a cada fila ---
def calcular_filas(row):
    tipos_first = [row["first_Type_1"], row["first_Type_2"]]
    tipos_second = [row["second_Type_1"], row["second_Type_2"]]

    row["first_type_advantage"] = calcular_ventaja(tipos_first, tipos_second)
    row["second_type_advantage"] = calcular_ventaja(tipos_second, tipos_first)
    return row

merged_df = merge_df.apply(calcular_filas, axis=1)

In [12]:
merged_df.to_csv("pokemon_battles_full.csv", index=False)

In [15]:
df = pd.read_csv("./data/pokemon_battles_full.csv")

#Eliminar columnas que no tienen función en este programa (informativas)
df.drop(columns=["first_Generation", "first_Legendary", "second_Generation", "second_Legendary"], inplace=True)

df.head()

,Winner,first_id,first_Name,first_Type_1,first_Type_2,first_HP,first_Attack,first_Defense,first_Sp_Atk,first_Sp_Def,...,second_id,second_Name,second_Type_1,second_Type_2,second_HP,second_Attack,second_Defense,second_Sp_Atk,second_Sp_Def,second_Speed
0,298,266,Larvitar,Rock,Ground,50,64,50,45,50,...,298,Nuzleaf,Grass,Dark,70,70,40,60,40,60
1,701,702,Virizion,Grass,Fighting,91,90,72,90,129,...,701,Terrakion,Rock,Fighting,91,129,90,72,90,108
2,668,191,Togetic,Fairy,Flying,55,40,85,80,105,...,668,Beheeyem,Psychic,NaN,75,75,75,125,95,40
3,683,237,Slugma,Fire,NaN,40,40,40,70,40,...,683,Druddigon,Dragon,NaN,77,120,90,60,90,48
4,151,151,Omastar,Rock,Water,70,60,125,115,70,...,231,Shuckle,Bug,Rock,20,10,230,10,230,5
